In [32]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sn
from pprint import pprint
import textwrap

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# PyTorch
import torch

# Hugging Face 
from transformers import pipeline, set_seed

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import transformermodels as tm
print(f'Package version: {tm.__version__}')
print(f'PyTorch version: {torch.__version__}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Package version: 0.0.post1.dev3+g8362a6c.d20240713
PyTorch version: 2.3.1+cu121


In [3]:
# GPU checks
is_cuda = torch.cuda.is_available()
print(f'CUDA available: {is_cuda}')
print(f'Number of GPUs found:  {torch.cuda.device_count()}')

if is_cuda:
    print(f'Current device ID:     {torch.cuda.current_device()}')
    print(f'GPU device name:       {torch.cuda.get_device_name(0)}')
    print(f'CUDNN version:         {torch.backends.cudnn.version()}')
    device_str = 'cuda:0'
    torch.cuda.empty_cache() 
else:
    device_str = 'cpu'
device = torch.device(device_str)
print()
print(f'Device for model training/inference: {device}')

CUDA available: True
Number of GPUs found:  1
Current device ID:     0
GPU device name:       NVIDIA GeForce RTX 3070 Laptop GPU
CUDNN version:         8902

Device for model training/inference: cuda:0


In [18]:
# Directories
data_dir = os.path.join(os.environ.get('HOME'), 'data', 'transformers')
textfile = os.path.join(data_dir, 'robert_frost.txt')
print(textfile)

/home/andreas/data/transformers/robert_frost.txt


In [24]:
lines = [line.rstrip() for line in open(textfile)]
lines = [line for line in lines if len(line) > 0]
display(lines[:10])

# Text generation pipeline
gen = pipeline(task='text-generation', device=device)
set_seed(1234)

['Two roads diverged in a yellow wood,',
 'And sorry I could not travel both',
 'And be one traveler, long I stood',
 'And looked down one as far as I could',
 'To where it bent in the undergrowth;',
 'Then took the other, as just as fair,',
 'And having perhaps the better claim',
 'Because it was grassy and wanted wear,',
 'Though as for that the passing there',
 'Had worn them really about the same,']

No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [25]:
print(lines[0])

Two roads diverged in a yellow wood,


In [28]:
pprint(gen(lines[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Two roads diverged in a yellow wood, then I went to the '
                    'point where my father and the driver were parked. At one '
                    'point they were sitting in the rear of the car (a small '
                    'car), and I got behind them and went for the'}]


In [29]:
pprint(gen(lines[0], num_return_sequences=3, max_length=20))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Two roads diverged in a yellow wood, only about 1.7 mi '
                    'from the base of Mt'},
 {'generated_text': 'Two roads diverged in a yellow wood, and in front of the '
                    'building a large boulder crashed and'},
 {'generated_text': 'Two roads diverged in a yellow wood, and as they '
                    'approached it many children were playing on the'}]


In [33]:
# Let's try to write a multi-line poem
def wrap(x):
    return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)

out = gen(lines[0], max_length=30)
print(wrap(out[0]['generated_text']))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Two roads diverged in a yellow wood, one on the top side, the other on
the bottom, and each side was divided by a road.


In [42]:
# Better is text that is not poetry
prompt = ('Neural networks with attention have been used with great success '
          'in natural language procesing')

out = gen(prompt, max_length=300)
print(wrap(out[0]['generated_text']))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Neural networks with attention have been used with great success in
natural language procesing in humans, humans in cats, goats,
chimpanzees, and humans in humans.  The results are now beginning to
show that there is a direct correlation between attention networks in
humans and cognitive performance in animals (Tau, 1994). In short, if
there are different attention networks in humans and animals on
different levels that influence human performance in terms of
attention (or not), then it must be inferred from the fact that there
are distinct cognitive function regions in humans, cats, humans, and
animals that must co-occur across different social networks in a
particular network environment.  These cognitive functions could
account for the lack of direct association between attention and
learning performance.

The results from the neuroimaging study
presented in Niebel and Küchen (2013) also show an indirect link
between attention in humans and cognitive function.  These results
indicat